# Importaciones y Funciones

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from keras import models, layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score
from sklearn.model_selection import KFold


In [ ]:
# Dicciónario para convertir los valores a categorías
LABELS = {0: "LowQuality", 1: "Average", 2: "HighQuality"}
BINS = [0, 4.5, 5.5, 10]

In [ ]:
# Función para cargar ambos datasets y convertir la columna quality a categorías
def cargar_datos(dataset):
    ds = pd.read_csv(dataset, sep=";")
    ds["quality_cat"] = pd.cut(ds["quality"], bins=BINS, labels=[0, 1, 2])
    return ds

In [ ]:
def categorizar(dataset):
    # crear una nueva columna con las categorías sin modificar los valores originales
    dataset["quality_cat"] = pd.cut(dataset["quality"], bins=BINS, labels=[0, 1, 2])
    return dataset

In [ ]:
def dividir_x_y(df, tipo="regresion"):
    if tipo == "categorico":
        x = df.drop(["quality", "quality_cat"], axis=1)
        y = df["quality_cat"]
    else:
        x = df.drop(["quality", "quality_cat"], axis=1)
        y = df["quality"]

    return x, y

In [ ]:
# Función para normalizar 
def normalizar(x, y):
    scaler = StandardScaler()
    x_scaled = scaler.fit_transform(x)
    y_onehot = keras.utils.to_categorical(y)
    return x_scaled, y_onehot


In [ ]:
# Función para configurar y ejecutar el modelo de clasificación
def clasificacion(X_scaled, y_onehot, fun_act):
    activation_fn = fun_act

    # Crear el modelo de clasificación con 3 capas de 64, 32 neuronas y 1 neurona por cada clase de salida
    classification_model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation=activation_fn, input_shape=(X_scaled.shape[1],)),
        tf.keras.layers.Dense(32, activation=activation_fn),
        tf.keras.layers.Dense(3, activation='softmax')
    ])

    # Compilar el modelo de clasificación con optimizador Adam, función de pérdidad Categorical Cross-Entropy, y de métrica de evaluación la exactitud
    classification_model.compile(optimizer='adam',
                                loss='categorical_crossentropy',
                                metrics=['accuracy'])

    # K-fold cross validation para el modelo de clasificación
    k = 5
    kf = KFold(n_splits=k, shuffle=True)

    acc = []
    loss = []

    # Entrenar el modelo de clasificación con 5-fold cross validation
    for train_index, val_index in kf.split(X_scaled):
        X_train, X_val = X_scaled[train_index], X_scaled[val_index]
        y_train, y_val = y_onehot[train_index], y_onehot[val_index]

        classification_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)  # type: ignore

        val_predictions = classification_model.predict(X_val)
        val_predictions = np.argmax(val_predictions, axis=1)
        val_true_labels = np.argmax(y_val, axis=1)

        acc.append(accuracy_score(val_true_labels, val_predictions))
        loss.append(classification_model.evaluate(X_val, y_val, verbose=0)[0])  # type: ignore

    print("\nPrecision con función de activación ", fun_act, ": ", np.mean(acc))
    print("Costo de perdida con función de activación ", fun_act, ": ", np.mean(loss))
    return np.mean(acc), np.mean(loss)

In [ ]:
# Función para configurar y ejecutar el modelo de regresión
def regresion(X_scaled, y, fun_act):
    activation_fn = fun_act

    # Crear el modelo de regresión con 3 capas de 64, 32 neuronas y 1 neurona de salida
    regression_model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation=activation_fn, input_shape=(X_scaled.shape[1],)),
        tf.keras.layers.Dense(32, activation=activation_fn),
        tf.keras.layers.Dense(1)
    ])

    # Compilar el modelo de regresión con optimizador Adam, función de pérdidad Error cuadrático medio, y de métricas de evaluación el error absoluto y cuadrático medio 
    regression_model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mse'])

    # K-fold cross validation para el modelo de regresión
    k = 5
    kf = KFold(n_splits=k, shuffle=True)

    mse_scores = []
    mae_scores = []

    # Entrenar el modelo de regresión con 5-fold cross validation
    for train_index, val_index in kf.split(X_scaled):
        X_train, X_val = X_scaled[train_index], X_scaled[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        regression_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0) # type: ignore
        
        val_predictions = regression_model.predict(X_val).flatten()
        mse = mean_squared_error(y_val, val_predictions)
        mae = mean_absolute_error(y_val, val_predictions)
        mse_scores.append(mse)
        mae_scores.append(mae)

    print("\nMSE con función de activación ", fun_act, ": ", np.mean(mse_scores))
    print("MAE con función de activación ", fun_act, ": ", np.mean(mae_scores))
    return np.mean(mse_scores), np.mean(mae_scores)

In [ ]:
# Esta función entrega tablas comparativas con las funciones Sigmoide y Tangente hiperbólica
def tabla_comparative(sigmoide, tanh, val1, val2):
    return pd.DataFrame(
        {
            "Función de activación": [val1, val2],
            "Sigmoid": [sigmoide[0], sigmoide[1]],
            "Tanh": [tanh[0], tanh[1]],
        }
    )

# Vino Blanco

In [ ]:
vino_blanco = cargar_datos("Data/winequality-white.csv")

## Modelo de clasificacion usando Red Neuronal

In [ ]:
x, y = dividir_x_y(vino_blanco, "categorico") # regresion o categorico
x_scaled, y_onehot = normalizar(x, y)

In [ ]:
clas_sigmoide = clasificacion(x_scaled, y_onehot, "sigmoid")

In [ ]:
clas_tanh = clasificacion(x_scaled, y_onehot, "tanh")

## Modelo de Regresion usando Red Neuronal

In [ ]:
x, y = dividir_x_y(vino_blanco, "regresion") # regresion o categorico
x_scaled, _ = normalizar(x, y)

In [ ]:
reg_sigmoide = regresion(x_scaled, y, "sigmoid")

In [ ]:
reg_tanh = regresion(x_scaled, y, "tanh")

## Análisis de los Resultados

### Modelo de Classification

Para el modelo de clasificación de vinos blancos, podemos ver que los resultados son dentro de todo similares, teniendo en cuenta esto, la funcion tangente hiperbólica (tanh) presenta un menor costo de pérdida a comparanción de la función sigmoidal. Lo que nos lleva a concluir que la función tangente hiperbólica sería la mas adecuada a implementar en este modelo. Se puede observar que la precisión (Accuracy) al utilizar la función tangente hiperbolica levemente mayor que la de función sigmoidal. Sin embargo, es importante considerar que esta métrica puede llegar a ser no la ideal al momento de realizar las evaluaciones ya que como sabemos gracias a los laboratorios anteriores, el dataset se encuentra desbalanceado.

In [ ]:
tabla_comparative(clas_sigmoide, clas_tanh, "Precisión", "Costo de perdida")

### Modelo de Regression
Para el modelo de regresión de vinos blancos podemos ver que los errores (cuadrático y absoluto medio), son dentro de todo similares, teniendo en cuenta esto, la funcion tangente hiperbólica (tanh) presenta errores mas pequeños a comparanción de la función sigmoidal. Lo que nos lleva a concluir que la función de activación de la tangente hiperbólica sería la mas adecuada a implementar en este modelo.

In [ ]:
tabla_comparative(reg_sigmoide, reg_tanh, "MSE", "MAE")

# Vino Tinto

In [ ]:
vino_tinto = cargar_datos("Data/winequality-red.csv")

## Modelo de clasificacion usando Red Neuronal


In [ ]:
x, y = dividir_x_y(vino_tinto, "categorico") # regresion o categorico
x_scaled, y_onehot = normalizar(x, y)

In [ ]:
clas_sigmoide = clasificacion(x_scaled, y_onehot, "sigmoid")

In [ ]:
clas_tanh = clasificacion(x_scaled, y_onehot, "tanh")

## Modelo de Regresion usando Red Neuronal

In [ ]:
x, y = dividir_x_y(vino_tinto, "regresion") # regresion o categorico
x_scaled, _ = normalizar(x, y)

In [ ]:
reg_sigmoide = regresion(x_scaled, y, "sigmoid")

In [ ]:
reg_tanh = regresion(x_scaled, y, "tanh")

## Análisis de los Resultados

### Modelo de Classification
Para el modelo de clasificación de vinos tintos, al igual que con los vinos blancos, la funcion tangente hiperbólica (tanh) presenta un menor costo de pérdida a comparanción de la función sigmoidal. Lo que nos lleva a concluir que la función tangente hiperbólica sería la mas adecuada a implementar en este modelo. Se puede observar que la precisión (Accuracy) al utilizar la función tangente hiperbolica levemente mayor que la de función sigmoidal. Sin embargo, es importante considerar que esta métrica puede llegar a ser no la ideal al momento de realizar las evaluaciones ya que como sabemos gracias a los laboratorios anteriores, el dataset se encuentra desbalanceado.

In [ ]:
tabla_comparative(clas_sigmoide, clas_tanh, "Precisión", "Costo de perdida")

### Modelo de Regression
Para el modelo de regresión de vinos tintos, al igual que en los vinos blancos, podemos ver que los errores (cuadrático y absoluto medio), son dentro de todo similares (esta vez mas existe menor diferencia entre ellos comparando esa diferencia entre errores de los vinos blancos). La funcion tangente hiperbólica (tanh) presenta errores mas pequeños a comparanción de la función sigmoidal. Esto nos lleva a concluir que la función de activación de la tangente hiperbólica sería la mas adecuada a implementar en este modelo.

In [ ]:
tabla_comparative(reg_sigmoide, reg_tanh, "MSE", "MAE")